In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats
from sklearn import metrics

In [ ]:
hab = pd.read_excel(path)
FC = pd.read_excel(path)
FR = pd.read_excel(path)
contB = pd.read_excel(path)

In [ ]:
def idx_of_the_nearest(data, value):
  '''
  Get nearest index
  value: Wanted timing
  '''
  idx = np.argmin(np.abs(np.array(data) - value))

  return idx

In [ ]:
def dFF_func(time, iso, green, BC_start_sec, BC_end_sec, BL_start_sec, BL_end_sec, fitting, mouse):
    '''
    Calculate dF/F
    time: time list
    iso: 405 data
    green: 470 data
    BC_start_sec: start time for fitting
    BC_end_sec: end time for fitting
    BL_start_sec: start time for baseline 
    BL_end_sec: end time for baseline
    fitting: 1: use 470 signal for fitting, 0: use 405 signal for fitting
    mouse: mouse number
    '''
    BC_start = idx_of_the_nearest(time_25min, BC_start_sec)
    BC_end = idx_of_the_nearest(time_25min, BC_end_sec)
    
    BL_start = idx_of_the_nearest(time_25min, BL_start_sec)
    BL_end = idx_of_the_nearest(time_25min, BL_end_sec)
    
    
    add1 = [1 for i in range(len(green))]
    
    fit = np.polyfit(time, iso, 1)
    fittedIso = fit[0] * np.array(time) + fit[1]
    isoBC = iso - fittedIso
    isoBCN = isoBC + add1
    
    fit = np.polyfit(time[BC_start:BC_end], green[BC_start:BC_end], 1)
    fittedGreen = fit[0] * np.array(time) + fit[1]
    
    if fitting == 1:
        greenBC = green - fittedGreen
        greenBCN = greenBC + add1
        
    else:
        greenBC = green - fittedIso
        greenBCN = greenBC + add1
        
    BL_mean = np.mean(greenBCN[BL_start:BL_end] / isoBCN[BL_start:BL_end])
    dFF = greenBCN / isoBCN - BL_mean
    
    fig_func(time, iso, green, fittedIso, fittedGreen, isoBCN, greenBCN, dFF, BC_start, BC_end, BL_start, BL_end, fitting, mouse)
    
    return dFF

In [ ]:
def fig_func(time, iso, green, fittedIso, fittedGeen, isoBCN, greenBCN, dFF, BC_start, BC_end, BL_start, BL_end, fitting, mouse):
    
    plt.figure(figsize=(10, 1))
    plt.subplots_adjust(wspace=0.4, hspace=0.4)
    
    plt.subplot(1, 3, 1)
    plt.plot(time, iso, color="gray")
    plt.plot(time, green, color="black")
    
    if fitting == 1:
        plt.plot(time[BC_start:BC_end], green[BC_start:BC_end], color="red")
        plt.plot(time, fittedIso, color="blue")
        
    if fitting == 1:
        plt.plot(time, fittedGeen, color="blue")
    else:
        plt.plot(time, fittedIso, color="blue")
        
    plt.title('{}'.format(mouse))

    plt.subplot(1, 3, 2)
    plt.plot(time, isoBCN, color="gray")
    plt.plot(time, greenBCN, color="black")

    plt.subplot(1, 3, 3)
    plt.plot(time, dFF)
    plt.plot(time[BL_start:BL_end], dFF[BL_start:BL_end], color='red')

# dF/F

## Habituation

In [ ]:
time_18min = [float(_) for _ in np.array(FC.filter(like=('time'), axis=1))][10:-9]
time_25min = [float(_) for _ in np.array(FR.filter(like=('time'), axis=1))][10:-9]

In [ ]:
mouse1_hab = np.array(hab.filter(['mouse1 iso', 'mouse1 green']))[10:-9]
mouse2_hab = np.array(hab.filter(['mouse2 iso', 'mouse2 green']))[10:-9]
mouse3_hab = np.array(hab.filter(['mouse3 iso', 'mouse3 green']))[10:-9]
mouse4_hab = np.array(hab.filter(['mouse4 iso', 'mouse4 green']))[10:-9]
mouse5_hab = np.array(hab.filter(['mouse5 iso', 'mouse5 green']))[10:-9]
mouse6_hab = np.array(hab.filter(['mouse6 iso', 'mouse6 green']))[10:-9]

In [ ]:
mouse1_hab_dFF = dFF_func(time_25min, mouse1_hab[:,0], mouse1_hab[:,1], 0, 590, 0, 30, 0, "mouse1")
mouse2_hab_dFF = dFF_func(time_25min, mouse2_hab[:,0], mouse2_hab[:,1], 0, 590, 0, 30, 0, "mouse2")
mouse3_hab_dFF = dFF_func(time_25min, mouse3_hab[:,0], mouse3_hab[:,1], 0, 500, 0, 30, 1, "mouse3")
mouse4_hab_dFF = dFF_func(time_25min, mouse4_hab[:,0], mouse4_hab[:,1], 0, 450, 0, 30, 1, "mouse4")
mouse5_hab_dFF = dFF_func(time_25min, mouse5_hab[:,0], mouse5_hab[:,1], 0, 590, 0, 30, 0, "mouse5")
mouse6_hab_dFF = dFF_func(time_25min, mouse6_hab[:,0], mouse6_hab[:,1], 0, 590, 0, 30, 1, "mouse6")

In [ ]:
hab_dFF = pd.DataFrame([mouse1_hab_dFF, mouse2_hab_dFF, mouse3_hab_dFF, mouse4_hab_dFF, mouse5_hab_dFF, mouse6_hab_dFF]).T
hab_dFF.columns = ["mouse1", "mouse2", "mouse3", "mouse4","mouse5", "mouse6"]
hab_dFF.index = time_25min

hab_dFF.to_excel('habituation_dFF.xlsx')

## Fear conditioning

In [ ]:
mouse1_FC = np.array(FC.filter(['mouse1 iso', 'mouse1 green']))[10:-9]
mouse2_FC = np.array(FC.filter(['mouse2 iso', 'mouse2 green']))[10:-9]
mouse3_FC = np.array(FC.filter(['mouse3 iso', 'mouse3 green']))[10:-9]
mouse4_FC = np.array(FC.filter(['mouse4 iso', 'mouse4 green']))[10:-9]
mouse5_FC = np.array(FC.filter(['mouse5 iso', 'mouse5 green']))[10:-9]
mouse6_FC = np.array(FC.filter(['mouse6 iso', 'mouse6 green']))[10:-9]

In [ ]:
mouse1_FC_dFF = dFF_func(time_18min, mouse1_FC[:,0], mouse1_FC[:,1], 0, 590, 0, 30, 1, "mouse1")
mouse2_FC_dFF = dFF_func(time_18min, mouse2_FC[:,0], mouse2_FC[:,1], 0, 590, 0, 30, 1, "mouse2")
mouse3_FC_dFF = dFF_func(time_18min, mouse3_FC[:,0], mouse3_FC[:,1], 0, 590, 0, 30, 1, "mouse3")
mouse4_FC_dFF = dFF_func(time_18min, mouse4_FC[:,0], mouse4_FC[:,1], 0, 500, 0, 30, 1, "mouse4")
mouse5_FC_dFF = dFF_func(time_18min, mouse5_FC[:,0], mouse5_FC[:,1], 0, 590, 0, 30, 1, "mouse5")
mouse6_FC_dFF = dFF_func(time_18min, mouse6_FC[:,0], mouse6_FC[:,1], 0, 590, 0, 30, 1, "mouse6")

In [ ]:
FC_dFF = pd.DataFrame([mouse1_FC_dFF, mouse2_FC_dFF, mouse3_FC_dFF, mouse4_FC_dFF, mouse5_FC_dFF, mouse6_FC_dFF]).T
FC_dFF.columns = ["mouse1", "mouse2", "mouse3", "mouse4","mouse5", "mouse6"]
FC_dFF.index = time_18min

FC_dFF.to_excel('FC_dFF.xlsx')

## Fear recall

In [ ]:
mouse1_FR = np.array(FR.filter(['mouse1 iso', 'mouse1 green']))[10:-9]
mouse2_FR = np.array(FR.filter(['mouse2 iso', 'mouse2 green']))[10:-9]
mouse3_FR = np.array(FR.filter(['mouse3 iso', 'mouse3 green']))[10:-9]
mouse4_FR = np.array(FR.filter(['mouse4 iso', 'mouse4 green']))[10:-9]
mouse5_FR = np.array(FR.filter(['mouse5 iso', 'mouse5 green']))[10:-9]
mouse6_FR = np.array(FR.filter(['mouse6 iso', 'mouse6 green']))[10:-9]

In [ ]:
mouse1_FR_dFF = dFF_func(time_25min, mouse1_FR[:,0], mouse1_FR[:,1], 0, 450, 0, 30, 1, "mouse1")
mouse2_FR_dFF = dFF_func(time_25min, mouse2_FR[:,0], mouse2_FR[:,1], 0, 590, 0, 30, 0, "mouse2")
mouse3_FR_dFF = dFF_func(time_25min, mouse3_FR[:,0], mouse3_FR[:,1], 200, 500, 0, 30, 1, "mouse3")
mouse4_FR_dFF = dFF_func(time_25min, mouse4_FR[:,0], mouse4_FR[:,1], 0, 590, 0, 30, 0, "mouse4")
mouse5_FR_dFF = dFF_func(time_25min, mouse5_FR[:,0], mouse5_FR[:,1], 0, 300, 0, 30, 1, "mouse5")
mouse6_FR_dFF = dFF_func(time_25min, mouse6_FR[:,0], mouse6_FR[:,1], 0, 590, 200, 230, 0, "mouse6")

In [ ]:
FR_dFF = pd.DataFrame([mouse1_FR_dFF, mouse2_FR_dFF, mouse3_FR_dFF, mouse4_FR_dFF, mouse5_FR_dFF, mouse6_FR_dFF]).T
FR_dFF.columns = ["mouse1", "mouse2", "mouse3", "mouse4","mouse5", "mouse6"]
FR_dFF.index = time_25min

FR_dFF.to_excel('FR_dFF.xlsx')

## Context B

In [ ]:
mouse1_contB = np.array(contB.filter(['mouse1 iso', 'mouse1 green']))[10:-9]
mouse2_contB = np.array(contB.filter(['mouse2 iso', 'mouse2 green']))[10:-9]
mouse3_contB = np.array(contB.filter(['mouse3 iso', 'mouse3 green']))[10:-9]
mouse4_contB = np.array(contB.filter(['mouse4 iso', 'mouse4 green']))[10:-9]
mouse5_contB = np.array(contB.filter(['mouse5 iso', 'mouse5 green']))[10:-9]
mouse6_contB = np.array(contB.filter(['mouse6 iso', 'mouse6 green']))[10:-9]

In [ ]:
mouse1_contB_dFF = dFF_func(time_25min, mouse1_contB[:,0], mouse1_contB[:,1], 0, 590, 0, 30, 1, "mouse1")
mouse2_contB_dFF = dFF_func(time_25min, mouse2_contB[:,0], mouse2_contB[:,1], 0, 590, 0, 30, 1, "mouse2")
mouse3_contB_dFF = dFF_func(time_25min, mouse3_contB[:,0], mouse3_contB[:,1], 0, 490, 0, 30, 1, "mouse3")
mouse4_contB_dFF = dFF_func(time_25min, mouse4_contB[:,0], mouse4_contB[:,1], 0, 590, 0, 30, 1, "mouse4")
mouse5_contB_dFF = dFF_func(time_25min, mouse5_contB[:,0], mouse5_contB[:,1], 0, 590, 0, 30, 1, "mouse5")
mouse6_contB_dFF = dFF_func(time_25min, mouse6_contB[:,0], mouse6_contB[:,1], 0, 590, 0, 30, 1, "mouse6")

In [ ]:
contB_dFF = pd.DataFrame([mouse1_contB_dFF, mouse2_contB_dFF, mouse3_contB_dFF, mouse4_contB_dFF, mouse5_contB_dFF, mouse6_contB_dFF]).T
contB_dFF.columns = ["mouse1", "mouse2", "mouse3", "mouse4","mouse5", "mouse6"]
contB_dFF.index = time_25min

contB_dFF.to_excel('contB_dFF.xlsx')

# Mean and SEM

In [ ]:
hab = hab_dFF.filter(["mouse1", "mouse2", "mouse3", "mouse4","mouse5", "mouse6"])
FC = FC_dFF.filter(["mouse1", "mouse2", "mouse3", "mouse4","mouse5", "mouse6"])
FR = FR_dFF.filter(["mouse1", "mouse2", "mouse3", "mouse4","mouse5", "mouse6"])
contB = contB_dFF.filter(["mouse1", "mouse2", "mouse3", "mouse4","mouse5", "mouse6"])

In [ ]:
habMean = hab.mean(axis=1)
FCMean = FC.mean(axis=1)
FRMean = FR.mean(axis=1)
contBMean = contB.mean(axis=1)

habSEM = hab.std(axis=1, ddof=1) / np.sqrt(len(hab.columns))
FCSEM = FC.std(axis=1, ddof=1) / np.sqrt(len(FC.columns))
FRSEM = FR.std(axis=1, ddof=1) / np.sqrt(len(FR.columns))
contBSEM = contB.std(axis=1, ddof=1) / np.sqrt(len(contB.columns))

# AUC and decay

## AUC

In [ ]:
def auc_func(signalData, event_start_ind, event_end_ind):
  '''
  Calculate Area Under Curve
  signal: data wanted to produce AUC
  event_start_ind: start point (index)
  event_end_ind: end point (index)
  '''

  aucDict = {}
  for i in range(len(signalData.columns)):
    signal = list(signalData.iloc[event_start_ind:event_end_ind,i])
    time = signalData.iloc[event_start_ind:event_end_ind,i].index

    auc = metrics.auc(time, signal)
    aucDict[signalData.columns[i]] = auc

  return aucDict

In [ ]:
ind_0sec = idx_of_the_nearest(time_18min, 0)
ind_1080sec = idx_of_the_nearest(time_18min, 1080)
ind_1500sec = idx_of_the_nearest(time_25min, 1500)

fast_start = idx_of_the_nearest(time_25min, 600)
fast_end = idx_of_the_nearest(time_25min, 600+420)

In [ ]:
habAUC_1500sec = auc_func(hab, ind_0sec, ind_1500sec)
FCAUC_1080sec = auc_func(FC, ind_0sec, ind_1080sec)
FRAUC_1500sec = auc_func(FR, ind_0sec, ind_1500sec)
contBAUC_1500sec = auc_func(contB, ind_0sec, ind_1500sec)

FRfastAUC = auc_func(FR, fast_start, fast_end)
FRlateAUC = auc_func(FR, fast_end, ind_1500sec)
contBfastAUC = auc_func(contB, fast_start, fast_end)
contBlateAUC = auc_func(contB, fast_end, ind_1500sec)

AUC = pd.DataFrame([habAUC_1500sec, FCAUC_1080sec, FRAUC_1500sec, contBAUC_1500sec, FRfastAUC, FRlateAUC, contBfastAUC, contBlateAUC], \
                        index=["hab_1500sec", "FC_1080sec", "FR_1500sec", "contB_1500sec", "FR_fast", "FR_late", "contB_fast", "contB_late"])

## decay

In [ ]:
decayTimeList = [_ * 0.05 for _ in range(11, 1000000)]

In [ ]:
def decay_func(data, start, end):
    '''
    Get decay
    start: start point (index)
    end: end point (index)
    '''
    
  peakDict = {}
  halfPeakDict = {}
  decayDict = {}

  for mouse in range(len(data.columns)):
    dataAnalysis = np.array(data.iloc[start:end, mouse])
    peakIndex = dataAnalysis.argmax()

    peakData = dataAnalysis[peakIndex:]
    peakFit = np.polyfit(decayTimeList[start:end][peakIndex:], peakData, 1)
    fittedPeak = peakFit[0] * np.array(decayTimeList) + peakFit[1]

    peakDict[data.columns[mouse]] = fittedPeak[start:end][peakIndex:][0]

    halfPeak = []
    c = 0
    for i in fittedPeak[start:][peakIndex:]:
      c += 1
      if i <= fittedPeak[start:end][peakIndex] / 2:
        halfPeak.append(i)
        halfPeakDict[data.columns[mouse]] = halfPeak[0]
        decayDict[data.columns[mouse]] = c*0.05
        break
    if len(halfPeak) == 0:
      halfPeakDict[data.columns[mouse]] = None
      peakDict[data.columns[mouse]] = None
      decayDict[data.columns[mouse]] = None


    #plt.figure(figsize=(2, 1))
    #plt.title(data.columns[mouse])
    #plt.plot(decayTimeList[start:end][peakIndex:], peakData, zorder=1)
    #plt.plot(decayTimeList[start:][peakIndex:peakIndex+c], fittedPeak[start:][peakIndex:peakIndex+c], zorder=2)
    #if len(halfPeak) == 0:
    #  pass
    #else:
    #  plt.scatter(decayTimeList[start:end][peakIndex] + c*0.05, halfPeak,color="red", zorder=3)

  return peakDict, halfPeakDict, decayDict

In [ ]:
shock1_peak, shock1_half_peak, shock1_decay = decay_func(FC, shock1_start, shock1_end)
shock2_peak, shock2_half_peak, shock2_decay = decay_func(FC, shock2_start, shock2_end)
shock3_peak, shock3_half_peak, shock3_decay = decay_func(FC, shock3_start, shock3_end)

In [ ]:
ind_610sec = idx_of_the_nearest(time_25min, 610)

In [ ]:
habPeak_1500sec, habHalfPeak_1500sec, habDecay_1500sec = decay_func(hab, ind_610sec, ind_1500sec)
contBPeak_1500sec, contBHalfPeak_1500sec, contBDecay_1500sec = decay_func(contB, ind_610sec, ind_1500sec)
FRPeak_1500sec, FRHalfPeak_1500sec, FRDecay_1500sec = decay_func(FR, ind_610sec, ind_1500sec)

In [ ]:
decay = pd.DataFrame([shock1_decay, shock2_decay, shock3_decay, habDecay_1500sec, FRDecay_1500sec, contBDecay_1500sec], \
                       index=["shock1", "shock2", "shock3", "hab_1500sec", "FR_1500sec","contB_1500sec"])